### Table of Contents
1. [Business understanding](#Businessunderstanding) 
     1. [Questions](#Questions)
      1. [Part I: What Year Have The Most Crime Rate?](#Questions_1)
       2. [Part II: what are the most committed crimes?](#Questions_2)
       3. [Part III: what districts have the most crimes?](#Questions_3)
       
2. [Data understanding](#Dataunderstanding)
3. [Data preparation](#clean)
4. [Data Analysis](#DataAnalysis)
5. [Results evaluation](#Results)


# Business understanding <a name = "Businessunderstanding" />
 
In this nootebook I am going to answer Three questions about austin crime data
1. What Year Have The Most Crime Rate? to see trends in the years 
2. what are the most committed crimes? this question can helps us to know the rate of crimes in austin city is it good to live in austin or not   
3. what districts have the most crimes? this question can help us identifies what distircts to avoid living in




In [105]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


%matplotlib inline

# Data Understanding  <a name = "Dataunderstanding" a>

### Acquire The Data
the data have been acquired from austin police using sodapy API [here](https://dev.socrata.com/consumers/getting-started.html)

In [106]:
def get_dataset(data, limit=200000):
            '''
            this function calls Socrata api to retrive the data 

            data  : csv dataset for austin crime data from 2015 to 2018 
            limit : limit of rows 
            '''
            from sodapy import Socrata

            client = Socrata("data.austintexas.gov", None)


            results = client.get(data, limit=200000)

            # Convert to pandas DataFrame
            results_df = pd.DataFrame.from_records(results)
            results_df.to_csv(f"{data}.csv")

#### Explore The Data 

Data consists of specific Part I crimes (murder, rape, robbery, aggravated assault, burglary, theft and auto theft)
Part II the date crime reported  the data of crime clearance status, clearance status case number and districts council, adp and crime describation

In [107]:
guid = pd.read_csv("data/GUIDE_-_Annual_Crime.csv")

In [108]:
guid

,Data purpose:,(none)
0,This Annual Crime dataset provides the raw dat...,NaN
1,Corresponding data:,NaN
2,This guide corresponds to the Annual Crime dat...,NaN
3,https://data.austintexas.gov/browse?Additional...,NaN
4,Note: The official crime counts used to produc...,NaN
5,Corresponding report:,NaN
6,The Annual Crime and Traffic Report is posted ...,NaN
7,http://www.austintexas.gov/page/annual-crime-t...,NaN
8,Field definitions:,NaN
9,Below are the fields and definitions contained...,NaN


### Field Name	Definition
- GO Primary Key	Case number => case_number  should be object 
- Council District	Austin city council district where the incident occurred
- GO Highest Offense Desc	The crime committed => crime describtion 
- Highest NIBRS/UCR Offense Description	The FBI category for the crime committed => crime type 
- GO Report Date	Date the incident was reported
- GO Location	Location of incident (rape locations are excluded to protect victims)
- Clearance Status	Codes used to identify if a crime was solved (see lookup table below)
- Clearance Date	Date the crime was solved
- GO District	APD Sector where the incident occurred (see lookup table below)
- GO Location Zip	Incident zip code
- GO Census Tract	Incident census tract
- GO X Coordinate	Incident location using projection NAD 1983 State Plane Texas Central FIPS 4203 (US Feet)
- GO Y Coordinate

- lookup table 
- Clearance Status	
  C 	 Cleared by Arrest
  O	     Cleared by Exception
  N	     Not cleared
- GO District	
  A	    Adam
  AP    Airport
  B	    Baker
  C	    Charlie
  D	    David
  E	    Edward
  F	    Frank
  G	    George
  H	    Henry
  I	    Ida
  UK    Unknown
      

```python
get data  2015 to 2018 
```

```python
get_dataset("spbg-9v94") # 2015 

get_dataset("8iue-zpf6") # 2016 

get_dataset("3t4q-mqs5") # 2017 

get_dataset("pgvh-cpyq") # 2018 
```

In [109]:
df_2015 = pd.read_csv("data/2015.csv")
df_2016 = pd.read_csv("data/2016.csv")
df_2017 = pd.read_csv("data/2017.csv")
df_2018 = pd.read_csv("data/2018.csv")

In [110]:
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38573 entries, 0 to 38572
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             38573 non-null  int64  
 1   go_primary_key                         38573 non-null  int64  
 2   council_district                       38401 non-null  float64
 3   go_highest_offense_desc                38573 non-null  object 
 4   highest_nibrs_ucr_offense_description  38573 non-null  object 
 5   go_report_date                         38573 non-null  object 
 6   go_location                            38092 non-null  object 
 7   clearance_status                       36649 non-null  object 
 8   clearance_date                         36649 non-null  object 
 9   go_district                            38573 non-null  object 
 10  go_location_zip                        38414 non-null  float64
 11  go

In [111]:
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37461 entries, 0 to 37460
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             37461 non-null  int64  
 1   go_primary_key                         37460 non-null  float64
 2   council_district                       37205 non-null  float64
 3   go_highest_offense_desc                37461 non-null  object 
 4   highest_nibrs_ucr_offense_description  37461 non-null  object 
 5   go_report_date                         37461 non-null  object 
 6   go_location                            36702 non-null  object 
 7   clearance_status                       35742 non-null  object 
 8   clearance_date                         35742 non-null  object 
 9   go_district                            37461 non-null  object 
 10  go_location_zip                        37294 non-null  float64
 11  go

In [112]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35097 entries, 0 to 35096
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             35097 non-null  int64  
 1   go_primary_key                         35095 non-null  float64
 2   council_district                       34844 non-null  float64
 3   go_highest_offense_desc                35097 non-null  object 
 4   highest_nibrs_ucr_offense_description  35097 non-null  object 
 5   go_report_date                         35097 non-null  object 
 6   go_location                            34286 non-null  object 
 7   clearance_status                       32667 non-null  object 
 8   clearance_date                         32665 non-null  object 
 9   go_district                            35097 non-null  object 
 10  go_location_zip                        34982 non-null  float64
 11  go

In [113]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37156 entries, 0 to 37155
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                37156 non-null  int64  
 1   go_primary_key_year_plus  37156 non-null  int64  
 2   council_district          36895 non-null  float64
 3   go_highest_offense_desc   37156 non-null  object 
 4   crime_type                37156 non-null  object 
 5   go_report_date            37156 non-null  object 
 6   go_location               36352 non-null  object 
 7   go_x_coordinate           36271 non-null  float64
 8   go_y_coordinate           36271 non-null  float64
 9   clearance_status          35702 non-null  object 
 10  clearance_date            35702 non-null  object 
 11  go_district               37156 non-null  object 
 12  go_location_zip           36989 non-null  float64
 13  go_census_tract           36989 non-null  float64
dtypes: flo

##### Basic stasstics 

In [114]:
# fisrt lets check the shape of the datasets 
print(f"""shape of 2015 data: {df_2015.shape[0]}
shape of 2016 data: {df_2016.shape[0]}
shape of 2017 data: {df_2017.shape[0]}
shape of 2018 data: {df_2018.shape[0]}""")

shape of 2015 data: 38573
shape of 2016 data: 37461
shape of 2017 data: 35097
shape of 2018 data: 37156


In [115]:
print(f"""avrege crimes in 2015 was: {df_2015.highest_nibrs_ucr_offense_description.count().mean()}
avrege crimes in 2016 was: {df_2016.highest_nibrs_ucr_offense_description.count().mean()}
avrege crimes in 2017 was: {df_2017.highest_nibrs_ucr_offense_description.count().mean()}
avrege crimes in 2018  was:  {df_2018.crime_type.count().mean()}""")

avrege crimes in 2015 was: 38573.0
avrege crimes in 2016 was: 37461.0
avrege crimes in 2017 was: 35097.0
avrege crimes in 2018  was:  37156.0


In [116]:
print(f"""precentge of crimes in 2015 was: {df_2015.highest_nibrs_ucr_offense_description .value_counts(normalize = True).mul(100)} 

precentge of crimes in 2016 was: {df_2016.highest_nibrs_ucr_offense_description .value_counts(normalize = True).mul(100)}

precentge of  crimes in 2017 was: {df_2017.highest_nibrs_ucr_offense_description .value_counts(normalize = True).mul(100)}

precentge of crimes in 2018  was:  {df_2018.crime_type.value_counts(normalize = True).mul(100)}""")

precentge of crimes in 2015 was: Theft          73.299977
Burglary       12.773183
Auto Theft      5.231639
Agg Assault     4.964613
Robbery         2.423975
Rape            1.246986
Murder          0.059627
Name: highest_nibrs_ucr_offense_description, dtype: float64 

precentge of crimes in 2016 was: Theft          70.048851
Burglary       13.803689
Agg Assault     5.811377
Auto Theft      5.405622
Robbery         2.802915
Rape            2.026107
Murder          0.101439
Name: highest_nibrs_ucr_offense_description, dtype: float64

precentge of  crimes in 2017 was: Theft          70.117104
Burglary       12.360031
Agg Assault     6.607402
Auto Theft      5.744081
Robbery         2.789412
Rape            2.310739
Murder          0.071231
Name: highest_nibrs_ucr_offense_description, dtype: float64

precentge of crimes in 2018  was:  Theft          71.514695
Burglary       11.225643
Auto Theft      6.531919
Agg Assault     5.727204
Robbery         2.750565
Rape            2.163850
Murder

In [117]:
print(f"""precentge of clearance status in 2015 was: {df_2015.clearance_status.value_counts(normalize = True).mul(100)} 

precentge of clearance status in 2016 was: {df_2016.clearance_status.value_counts(normalize = True).mul(100)}

precentge of clearance status crimes in 2017 was: {df_2017.clearance_status.value_counts(normalize = True).mul(100)}

precentge of clearance status in 2018  was:  {df_2018.clearance_status.value_counts(normalize = True).mul(100)}""")

precentge of clearance status in 2015 was: N    83.118230
C    14.311441
O     2.570329
Name: clearance_status, dtype: float64 

precentge of clearance status in 2016 was: N    83.137485
C    14.025516
O     2.836998
Name: clearance_status, dtype: float64

precentge of clearance status crimes in 2017 was: N    83.423639
C    14.001898
O     2.574464
Name: clearance_status, dtype: float64

precentge of clearance status in 2018  was:  N    86.070808
C    13.416615
O     0.512576
Name: clearance_status, dtype: float64


# Data preparation <a name ="clean"/>
in this dataset I have to change some columns names/ data type drop some and impute the null values 

##### first removeing Unamed: 0  because it's appears after saving the data no need for it it's index for the rows

In [118]:
df_2015.drop("Unnamed: 0", axis=1, inplace = True)
df_2016.drop("Unnamed: 0", axis=1, inplace = True)
df_2017.drop("Unnamed: 0", axis=1, inplace = True)
df_2018.drop("Unnamed: 0", axis=1, inplace = True)

#### changing columns name to concat the data later
#### I need to make all columns with same name so we would not have problem concating the data 

In [119]:
def change_col_name(df, column, new_name):
    '''
    this function change columns name 
    df : dataframe 
    column: the name of the column 
    new_name: the new name for the column 
    '''
    df.rename(columns={column : new_name}, inplace=True)
    


In [120]:
change_col_name(df_2015, "go_primary_key", "case_number")
change_col_name(df_2016, "go_primary_key", "case_number")
change_col_name(df_2017, "go_primary_key", "case_number")
change_col_name(df_2018, "go_primary_key_year_plus", "case_number")

In [121]:
change_col_name(df_2018, "go_highest_offense_desc", "crime_desc")
change_col_name(df_2017, "go_highest_offense_desc", "crime_desc")
change_col_name(df_2016, "go_highest_offense_desc", "crime_desc")
change_col_name(df_2015, "go_highest_offense_desc", "crime_desc")

change_col_name(df_2017, "highest_nibrs_ucr_offense_description", "crime_type")
change_col_name(df_2016, "highest_nibrs_ucr_offense_description", "crime_type")
change_col_name(df_2015, "highest_nibrs_ucr_offense_description", "crime_type")

##### fix column order so they all match 

In [122]:
df_2018 = df_2018[['case_number',
 'council_district',
 'crime_desc',
 'crime_type',
 'go_report_date',
 'go_location',
 'clearance_status',
 'clearance_date',
 'go_district',
 'go_location_zip',
 'go_census_tract',
 'go_x_coordinate',
 'go_y_coordinate']]

##### concat the data  

In [123]:
df = pd.concat([df_2018, df_2017, df_2016, df_2015], axis = 0)

##### checking null values

In [124]:
df.isnull().sum().sort_values(ascending = False)

clearance_date      7529
clearance_status    7527
go_y_coordinate     3210
go_x_coordinate     3210
go_location         2855
council_district     942
go_census_tract      608
go_location_zip      608
case_number            3
go_district            0
go_report_date         0
crime_type             0
crime_desc             0
dtype: int64

#### fix date columns  convert it to date time  and get year month and day 


In [125]:
df.go_report_date =  pd.to_datetime(df.go_report_date)
df.clearance_date =  pd.to_datetime(df.clearance_date)

In [126]:
df["report_year"] = df.go_report_date.dt.year 
df["report_month"] = df.go_report_date.dt.month_name() 
df["report_day"] = df.go_report_date.dt.day_name() 


In [127]:
 df["clearance_year"] = df.clearance_date.dt.year
df["clearance_month"] = df.clearance_date.dt.month_name()
df["clearance_day"] = df.clearance_date.dt.day_name()

1. drop uncessry columns i don't need theses columns for my analysis go_x_coordinate, go_y_coordinate i can use zip code instaed of them 
2.  case number have no info I can get from it as they serve as primary key only 
3.  go_census_tract can't help me in my analysis 


In [128]:
df.drop("case_number", axis=1, inplace=True)
df.drop("go_census_tract", axis=1, inplace=True)
df.drop("go_x_coordinate", axis= 1, inplace=True)
df.drop("go_y_coordinate", axis= 1, inplace=True)

#### dealing with null values

##### the null values in go_location are excluded to protect rape victims
##### so I imputed the null value with Excluded


In [129]:
df.go_location.fillna("Excluded", inplace = True)

####  for clearance status and date 

#### I will impute then with N = Not cleared because I think it's never solved 

In [130]:
df.clearance_status.fillna("N", inplace = True)
df.clearance_date.fillna("Not cleared", inplace = True)
df.clearance_year.fillna("Not cleared", inplace = True)
df.clearance_month.fillna("Not cleared", inplace = True)
df.clearance_day.fillna("Not cleared", inplace = True)

##### here I amgoing to solve null values in council district and zip location

<center><title> council district <title><center>
<img src = "Images/Screenshot 2021-05-09 234406.jpg">

<center><title> go district <title><center>
<img src = "Images/Screenshot 2021-05-10 021353.jpg">

<center><title> zip location <title><center>
<img src = "Images/Screenshot 2021-05-11 084116.jpg">

##### with this function I am going to fill null values for zip_location using geopy then I will use it to fill null values in districts

In [131]:
def fill_zipcode(location_name):
    '''
    this function return zip location if it's found 
    otherwise return Excluded for Excluded location or UNKNOWN for UNKNOWN locations in the dataset 
    
    location_name: name of the location 
    '''

    try:
        if location_name ==  "Excluded":
            return "Excluded"
            
        
        elif location_name == "UNKNOWN":
            return "UNKNOWN"
        
        else:
            from geopy.geocoders import Nominatim
            geolocator = Nominatim(user_agent="geoapiExercises")
            place = location_name
            location = geolocator.geocode("%s, Austin" % place, country_codes="US")
            # traverse the data
            
            data = location.raw
            loc_data = data['display_name'].split()
            return loc_data[-3].split(",")[0]
            

    except Exception as e:
        print(e)

#### maps some zip code to districts 


In [132]:
district_to_zip = {
    78746.0:"WEST LAKE",
    78660.0:"PFLUGERVILLE",
    78744.0: "2",
    78653.0: "MANOR",
    78735.0: "8",
    78748.0: "5",
    78724.0: "1",
    78728.0: "7",
    78729.0: "6",
    78754.0: "1",
    78664.0: "Round Rock",
    78747.0: "2",
    78617.0: "2",
    78681.0: "Round Rock",
    78732.0: "6",
    78719.0: "2",
    78749.0: "8",
    78726.0: "6",
    78734.0: "Lakeway",
    78730.0: "10",
    78652.0: "Manchaca",
    78613.0: "Cedar Park",
    78745.0: "2",
    78717.0:"6",
    78736.0: "Austin Zoo"
}

#### Impute the null values in zip location 

In [162]:
df["go_location_zip"] = df.apply(lambda x: fill_zipcode(x["go_location"]) if pd.isnull( x["go_location_zip"]) else x['go_location_zip'], axis=1)


##### maps districts accourding to zip location 

In [163]:
df.loc[:, "council_district"] = df.apply(
    lambda x: district_to_zip.get(x["go_location_zip"], x["council_district"]), axis=1
)

#### the remaining null values in zip location and council districts will be imputed with UNKNOWN
#### because geopy can't idnetfies them using the maps so will make it UNKNOWN and also for districts 

In [164]:
df.council_district.fillna("UNKNOWN", inplace = True)
df.go_location_zip.fillna("UNKNOWN", inplace = True)

##### here I am going to fix the maping so that 1.0 is the same as 1 and so on 

In [165]:
fix_distircts = { 

    "1.0":"1",
    "2.0":"2",
    "3.0":"3",
    "4.0":"4",
    "5.0":"5",
    "6.0":"6",
    "7.0":"7",
    "8.0":"8",
    "9.0":"9",
    "10.0":"10",
    "Austin Zoo":     "Austin Zoo",
    "Cedar Park" :      "Cedar Park",
    "Lakeway" :          "Lakeway",
    "MANOR"   :          "MANOR",
    "Manchaca" :          "Manchaca",
    "PFLUGERVILLE" :    "PFLUGERVILLE",
    "Round Rock"   :      "Round Rock",
    "UNKNOWN"     :      "UNKNOWN",
    "WEST LAKE"    :    "WEST LAKE"
}

In [166]:
df.council_district = df.council_district.map(fix_distircts)

#### dealing with clearance_status, go_district
#### here I will cahnge the shortcut to the full info 

In [167]:
dict_clearnce_status = {
"C":"Cleared by Arrest",
"O":"Cleared by Exception",
"N":"Not cleared"
}

dict_go_district = {"A":"Adam",
"AP":"Airport",
"B":"Baker",
"C":"Charlie",
"D":"David",
"E":"Edward",
"F":"Frank",
"G":"George",
"H":"Henry",
"I":"Ida",
"UK":"UNKNOWN",
 "88": "88"}

##### maps the names of dict to columns 

In [168]:
df.go_district = df.go_district.map(dict_go_district)
df.clearance_status = df.clearance_status.map(dict_clearnce_status)

#### saving the data 

In [169]:
# df.to_csv("data/clean_data.csv")

df = pd.read_csv("data/clean_data.csv")

In [170]:
df.head()

,Unnamed: 0,council_district,crime_desc,crime_type,go_report_date,go_location,clearance_status,clearance_date,go_district,go_location_zip,report_year,report_month,report_day,clearance_year,clearance_month,clearance_day
0,0,9.0,AUTO THEFT,Auto Theft,2018-12-10,2902 HAMPTON RD,Not cleared,2018-12-19 00:00:00,Baker,78705.0,2018,December,Monday,2018.0,December,Wednesday
1,1,9.0,THEFT BY SHOPLIFTING,Theft,2018-10-16,1201 BARBARA JORDAN BLVD,Not cleared,2018-10-17 00:00:00,Baker,78723.0,2018,October,Tuesday,2018.0,October,Wednesday
2,2,10.0,BURGLARY OF VEHICLE,Theft,2018-06-22,2600 LAKE AUSTIN BLVD,Not cleared,2018-06-27 00:00:00,Baker,78703.0,2018,June,Friday,2018.0,June,Wednesday
3,3,4.0,BURGLARY OF RESIDENCE,Burglary,2018-07-05,5330 N IH 35 SVRD SB,Not cleared,2018-09-13 00:00:00,Ida,78751.0,2018,July,Thursday,2018.0,September,Thursday
4,4,7.0,BURGLARY OF VEHICLE,Theft,2018-09-24,12802 POWDERHORN ST,Not cleared,2018-09-24 00:00:00,Adam,78727.0,2018,September,Monday,2018.0,September,Monday


# Data Analysis <a name ="DataAnalysis" />

### Questions <a name = "Questions" a>

### Part I: What Year Have The Most Crime Rate? <a name = "Questions_1" a>

In [171]:
crime_per_year = pd.crosstab(df.crime_type, df.report_year).mean()
values = np.sort(crime_per_year.values)

In [172]:
import plotly.express as px
def bar_plot(title, labels, x, y, data ):
        fig = px.bar(data, x=x, y=y, title = title, labels = labels, text = y)
        fig.update_traces(marker_color= '#00a2ed',  textposition='inside', texttemplate='%{text:.2s}')
        fig.update_xaxes(showgrid=False)
        fig.update_layout(template="plotly_white")


        fig.show()
bar_plot("Crime rate from (2015 - 2018", dict(report_year = "Year", y =  "Crime Rate"),crime_per_year.index, values, crime_per_year )

In [173]:
fig = px.scatter(crime_per_year, x=crime_per_year.index, y= values, title='Crime rate from (2015 - 2018)')
fig.update_traces(marker_color= '#00a2ed',  mode='lines+markers')
fig.update_xaxes(showgrid=False)
fig.update_layout(template="plotly_white")
fig.show()

In [174]:
top_18 = df.query("report_year =='2018'")["crime_type"].value_counts()
top_18


Theft          26572
Burglary        4171
Auto Theft      2427
Agg Assault     2128
Robbery         1022
Rape             804
Murder            32
Name: crime_type, dtype: int64

In [175]:
top_17 = df.query("report_year =='2017'")["crime_type"].value_counts()
top_17


Theft          24609
Burglary        4338
Agg Assault     2319
Auto Theft      2016
Robbery          979
Rape             811
Murder            25
Name: crime_type, dtype: int64

In [176]:
bar_plot("Crime rate for 2018", dict(index = "crime type", y =  "Crime Rate"), top_18.index, top_18.values, top_18 )


### Part II: what are the most committed crimes? <a name = "Questions_2" a>


In [177]:
crimes = df.crime_type.value_counts()
bar_plot("Crimes (2015 - 2018)", dict(index = "crime type", y =  "Crime Rate"), crimes.index, crimes.values, crimes)


In [178]:
crimes

Theft          105696
Burglary        18607
Agg Assault      8539
Auto Theft       8486
Robbery          3986
Rape             2855
Murder            118
Name: crime_type, dtype: int64

In [179]:
crime_per_districs = pd.crosstab(df.crime_type, df.council_district).mean().sort_values(ascending = False)
crime_per_districs

council_district
9.0             3258.000000
4.0             2840.142857
3.0             2797.571429
2               2711.000000
7.0             2496.285714
1.0             1554.428571
10.0             921.857143
5                759.285714
6                619.857143
8                615.571429
1                542.714286
5.0              542.428571
WEST LAKE        488.285714
6.0              420.142857
Cedar Park       199.142857
PFLUGERVILLE     114.714286
UNKNOWN           86.857143
8.0               80.857143
Austin Zoo        54.428571
10                32.857143
MANOR             24.428571
2.0                7.857143
7                  7.000000
Manchaca           6.714286
Round Rock         1.000000
Lakeway            0.428571
dtype: float64

### Part III: what districts have the most crimes? <a name = "Questions_3" a>

In [180]:
bar_plot("Rate of Crime in Districts", dict(index = "Council District", y =  "Crime Rate"), crime_per_districs.index, np.sort(crime_per_districs.values)[::-1], crime_per_districs)

In [181]:
df.query("council_district == '1'")["crime_type"].value_counts()

Theft          2317
Burglary        645
Agg Assault     406
Auto Theft      228
Rape            105
Robbery          90
Murder            8
Name: crime_type, dtype: int64

In [182]:
df.query("council_district == '10'")["crime_type"].value_counts()

Theft          174
Burglary        29
Agg Assault     12
Auto Theft      10
Robbery          3
Rape             2
Name: crime_type, dtype: int64

In [183]:
df.query("council_district == '2'")["crime_type"].value_counts()

Theft          12622
Burglary        2783
Auto Theft      1404
Agg Assault     1343
Robbery          406
Rape             396
Murder            23
Name: crime_type, dtype: int64

In [184]:
df.query("council_district == '3'")["crime_type"].value_counts()

Series([], Name: crime_type, dtype: int64)

In [185]:
df.query("council_district == '4'")["crime_type"].value_counts()

Series([], Name: crime_type, dtype: int64)

In [186]:
df.query("council_district == '5'")["crime_type"].value_counts()

Theft          4008
Burglary        637
Auto Theft      254
Agg Assault     242
Rape             91
Robbery          83
Name: crime_type, dtype: int64

In [187]:
df.query("council_district == '6'")["crime_type"].value_counts()

Theft          3180
Burglary        575
Agg Assault     232
Auto Theft      199
Rape             77
Robbery          75
Murder            1
Name: crime_type, dtype: int64

In [188]:
df.query("council_district == '8'")["crime_type"].value_counts()

Theft          3306
Burglary        534
Auto Theft      182
Agg Assault     181
Rape             60
Robbery          45
Murder            1
Name: crime_type, dtype: int64

In [189]:
df.query("council_district == '9'")["crime_type"].value_counts()

Series([], Name: crime_type, dtype: int64)

# Results evaluation <a name ="Results"/>

In this notebook, we took a look at crime in Austin from 2015 to 2018
We identified which year has the most crimes and what was the increase and decrease in the crimes, we found out that 2018 has the most crime.
then We take look at what are the most committed crimes.
Finally, we identified what districts have the most crimes. We found that districts 9, 4, and 3 are at the top three.
The findings here are observational, not the result of a formal study.